
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Sieci rekurencyjne</big></big></big></big></big>

---

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Rekurencyjne sieci neuronowe
<img src="../nn_figures/rnn-diagrams.jpeg" width="90%">

1. obraz w klasyfikację
2. klasyfikacja obraz przez sekwencję; wiele klasyfikacji
3. sekwencja wyrazów w analizę (np. sentymentu)
  * generacja obrazu z sekwencji
4. translacja: sekwencja w sekwencję, zwykle różnej długości
5. klasyfikacja każdej ramki

Sieć rekurencyjna używa pewnego rekurencyjnego wzoru w każdym kroku 
$$\begin{align}
h_t&=f(h_{t-1}, x_t; U,V,W)\\
h_t&=\tanh(Wh_{t-1}+Ux_t)\\
y_t&=Vh_t
\end{align}$$
%![rnn-diags.jpeg](attachment:rnn-diags.jpeg)

# Rozwinięcie sieci rekurencyjnej
<img src="../nn_figures/rnn.jpeg" width="90%"> [Nature]

1. __rozwinięcie sieci__ w $n$ składowych dla wygenerowania $n$-elementowego ciągu
2. $x_t$ to wejscie w chwili $t$
  * gdy generujemy więcej niż jedno słowo (znak) wprzód, to poprzednio wygenerowane staje się wejściem do następnego
  * oczywiście może być tylko jedno $x$
3. __pamięć__ (stan ukryty)) $h_t$ 
  * obliczane na podstawie poprzednich $h_t=f(Ux_t+Ws_{t-1})$
  * $f$ funkcją nieliniową
4. __wyjście__ $y_t$ w chwili $t$
  * zwykle jako $softmax(Vh_t)$
    * z tego stan wyjściowy wybierany przez $\arg\max$ albo sampling
  * zwraca wektor prawdopodobieństw stanów dyskretnych
  * interesujący może być np. tylko ostatni stan określający znaczenie zdania (sentiment analysis)
5. $h_t$ przechowuje __całą__ informację na temat poprzednich stanów obliczeń
  * własność Markowa
  * praktycznie nie jest wystarczająca
6. __wszystkie__ kroki __dzielą__ te same parametry $U, V, W$

### Sieci głębokie RNN
2. Model u góry to oczywiscie model płytki
1. ten model może był także __deep__
  * może być więcej niż jedna warstwa ukryta
    * każda z nich aktualizuje swój stan
  * warstwa ukryta może aktualizować swój stan przez wielokrokowe przeliczenia (głęboka sieć)
  * też kombinacje obu podejść

## Problemy RNN
1. od dawna znane różne podstawowe architektury RNN
  * stanem pamięci jest stan ukryty i tam następuje rekurencja
  * aktualny stan wyjsciowy staje się _dodatkowym_ stanem wejściowym (jak w automatach)
2. podstawowymi problemami są
  * pamięć jedynie ostatnich akcji, _zapominanie_ stanów poprzednich
  * pamięć jedynie pojedynczych stanów globalnych dla całego modelu bez pamięci stanów ostatnich
  * stąd potrzeba modelu wypełniającego tą dziurę - __long-short time memory__
  * eksplodujące / zanikające gradienty

## Zastosowania
1. __modelowanie i generowanie języka__
  * predykcja __prawdopodobieństwa__, że zdanie jest poprawne
  * samplując z tego dostajemy model __generatywny__
  * model językowy z użyciem __n-gramów__
  $$P(w_1,\dots,w_m)=\prod_{i=1}^mP(w_i\mid w_1,\dots,w_{i-1})\approx\prod_{i=1}^mP(w_i\mid w_{i-(n-1)},\dots,w_{i-1})$$
  dla n-gramów $$P(w_i\mid w_{i-(n-1)},\dots,w_{i-1})=\frac{\#(w_{i-(n-1)},\dots,w_{i-1}, w_i)}{\#(w_{i-(n-1)},\dots,w_{i-1})}$$
2. __tłumaczenie języka__
  * podobne do modelowania
  * wymaga zwykle przeczytania kompletnego zdania w jednym języku __przed__ wygenerowaniem pierwszego słowa nowego zdania
3. __rozpoznawanie języka__
  * wejściem są odczytane __fonemy__
  * wyjściem nowe fonemy lub transkrypcja na zdania (tłumaczenie)
4. Modele RNN pozwalają przyjmować wejścia o __zmiennej długości__
  * na przykład opis obrazu jako wiele losowych sampli z niego

<img src="../nn_figures/rnn-diagrams.jpeg" width="80%"> [Karpathy]


Koszt
1. many-to-many: 
  * w każdym kroku można obliczyć koszt i gradient, 
  * po końcu sekwencji zsumować
  * zaaplikować zmiany
2. many-to-one
  * wartość znana jest dopiero po końcu całej sekwencji
  * koszt i gradient na końcu
3. one-to-many
  * znowu koszt na każdym kroku
4. sekwencja do sekwencji to złożenie many-to-one oraz one-to-many
  * _enkoder_ tworzy reprezentację całego wejścia (np. zdania w jednym języku)
  * _dekoder_ odtwarza ten ukryty stan do postaci sekwencji
  * oczywiście kompresja do jednego stanu jest trudna
    * Macron mówił w przemówieniu "Mes cher compatriots"
    * co w automatycznym tłumaczeniu na angielski (czytaj amerykański) wyszło "my fellow americans" :-))

## Back-Propagation Through Time BPTT
1. za każdym razem patrzymy kilka kroków wstecz

In [1]:
# klasa RNN (za http://wildml.com)
class RNNNumpy():
    def __init__(self, word_dim, hidden_dim=100, bptt_truncate=4):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), 
                                   (hidden_dim, word_dim))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), 
                                   (word_dim, hidden_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), 
                                   (hidden_dim, hidden_dim))
        
    def forward_propagation(self, x):
        # The total number of time steps
        T = len(x)
        # During forward propagation we save all hidden states in s because need them later.
        # We add one additional element for the initial hidden, which we set to 0
        s = np.zeros((T + 1, self.hidden_dim))
        s[-1] = np.zeros(self.hidden_dim)
        # The outputs at each time step. Again, we save them for later.
        o = np.zeros((T, self.word_dim))
        # For each time step...
        for t in np.arange(T):
            # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
 
    def predict(self, x):
        # Perform forward propagation and return index of the highest score
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
        # or sample
        return np.random.choice(range(len(o)), size=1, p=o)
 
    #RNNNumpy.predict = predict
    #RNNNumpy.forward_propagation = forward_propagation


## BPTT
<img src="../nn_figures/rnn.jpeg" width="70%"> [Nature]
1. w każdym kroku należy znaleźć wszystkie macierze parametrów $U, V, W$
  * są wspólne dla wszystkich kroków
  * zwykle mają dużo parametrów
  * niech będzie $N$ różnych słów, a pamięć jest reprezentowana przez wektor o długosci $K$
    * $x_t\in\mathbb{R}^{N}$
    * $o_t\in\mathbb{R}^{N}$
    * $s_t\in\mathbb{R}^{K}$
    * $U\in\mathbb{R}^{K\times{}N}$
    * $V\in\mathbb{R}^{N\times{}K}$
    * $W\in\mathbb{R}^{K\times{}K}$
2. parametry są __dzielone__ we wszystkich przewidywanych krokach
  * gradient w aktualnym kroku zależy 
    * od obliczeń w aktualnym kroku czasu
    * od obliczeń w poprzednim kroku
  * odpowiada to wykorzystaniu __reguły łańcuchowej__

In [2]:
# za [Britz]
def bptt(self, x, y):
    T = len(y)
    # wykonanie propagacji wprzód (zwraca ostatnie wyjscie i stan pamięci)
    #  forward_propagation() wykonuje kroki wprzód zapamiętując wszystkie wartosci pośrednie,
    #  które będą później potrzebne
    o, s = self.forward_propagation(x)
    # macierze potrzebne dla akumulacji gradientów
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # teraz cofając się wstecz w obliczeniach
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # wstęczne obliczenia dla ostatniego kroku
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # wsteczna propagacja w czasie po poprzedzających krokach, ale co najwyżej bptt_truncate kroków
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # aktualizacja
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

## BPTT
1. Algorytm jest w stanie nauczyć się prostych zależności
  * kolejność słów: bi-gramy, tri-gramy
  * częstość występowania słów
  * prostej składni
  * prostej interpunkcji
3. Jednak
  * podawane zdania są zbyt krótkie by nauczyć poprawnej gramatyki
  * dłuższe zdania znacznie zwiększają złożoność uczenia
  * __nie jest w stanie__ nauczyć się zależności między __odległymi__ słowami
    * proste RNN są w stanie imitować __jedynie__ pamięć krótko-terminową
  * BPTT cierpi w dużym stopniu na problem zanikającego / eksplodującego gradientu

## Teacher forcing
W modelu many-to-many, np w przewidywaniu kolejnego słowa, zamiast przekazywać do następnego stanu tylko stan ukryty, możemy
1. przekazywać wektor (ew. znormalizowanych) logitsów predykcji $o_{t-1}$ do następnego stanu,
2. __teacher forcing__
  * logitsy są zwykle bardzo rozmyte, bo jest wiele wyjściowych słów, stąd sygnał propagujący jest słaby
  * w trakcie trenowania można wybrać odpowiedź $y_{t-1}$ i ją przekazać (jako wektor one-hot)
  * w trakcie testowania już tylko wektor $o_{t-1}$
3. teacher forcing czasem zachowuje się niestabilnie
  * rzadka odpowiedź na początku powoduje cżasto dziwne decyzje w kolejnych krokach
4. __schedula sampling__
  * algorytm uczony jest przewidywania _kilku_ obiektów w przód
  * przekazywanie jednej odpowiedzi $y_{t-1}$ nie powoduje niestabilności, nawet jeśli rzadka
5. __professor forcing__
  * ukryte neurony stanów są regularyzowane
  * też zmniejsza niestabilność

### BPTT koszt i wsteczna propagacja
<img src="../nn_figures/rnn.jpeg" width="70%"> [Nature]

<img src="../nn_figures/rnn-bptt1.png" width="70%"> [Nature]

1. koszt
$$E(y, \widehat{y})=\sum_tE_t(y_t,\widehat{y}_t)$$
2. dla $z_3=Vs_3$ mamy
$$\begin{align}
\frac{\partial E_3}{\partial V} &= \frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial\widehat{y}_3}{\partial V}\\
&=\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial\widehat{y}_3}{\partial z_3} \frac{\partial z_3}{\partial V}\\
&=(\widehat{y}_3-y_3)\otimes s_3
\end{align}$$
3. dla pochodnej po $W$ zaczyna się pojawiać rekurencja
$$\begin{align}
\frac{\partial E_3}{\partial W} &= \frac{\partial E_3}{\partial s_3} \frac{\partial s_3}{\partial W}\\
&= \frac{\partial E_3}{\partial \widehat{y}_3}\frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial W}\\
&\hskip3em\text{jednak $s_3$ bezpośrednio zależy od $s_2$, które nie jest stałe!}\\
s_3&=\tanh(U x_t+W s_2)\\
\frac{\partial E_3}{\partial W} &=\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial s_t}\frac{\partial s_t}{\partial W}\\
\end{align}$$
<img src="../nn_figures/rnn-bptt-gradients.png" width="70%"> [Nature]
4. w rzeczywistości BPTT niewiele się różni od zwykłej wstecznej propagacji
  * w sieci warstwowej parametry między warstwami __nie są__ dzielone
  * nie ma potrzeby ich sumowania
  * w analogiczny sposób można zdefiniować regułę delta
  $$\delta^3_2=\frac{\partial E_3}{\partial z_2}=\frac{\partial E_3}{\partial s_3}\frac{\partial s_3}{\partial s_2}\frac{\partial s_2}{\partial s_2}$$

### BPTT i zanikający gradient
1. podstawowym problemem w uczeniu jest zanikanie gradientu
  * problem zauważył Hochreiter, który był autorem modelu LSTM
$$\frac{\partial E_3}{\partial W} =\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \frac{\partial s_3}{\partial s_t}\frac{\partial s_t}{\partial W}$$
2. w rozwiązaniu występuje czynnik
$$\frac{\partial s_3}{\partial s_t}$$
  * i tak chociażby $$\frac{\partial s_3}{\partial s_1} = \frac{\partial s_3}{\partial s_2}\frac{\partial s_2}{\partial s_1}$$
  * skąd mamy
  $$\frac{\partial E_3}{\partial W} =\sum_{t=0}^3\frac{\partial E_3}{\partial \widehat{y}_3} \frac{\partial \widehat{y}_3}{\partial s_3} \left(\prod_{j=t+1}\frac{\partial s_j}{\partial s_{j-1}}\right)\frac{\partial s_t}{\partial W}$$
  * $s_t=\tanh(Ux_t+Ws{t-1})$
  * $\tanh()$ ma obszar saturacji po lewej i prawej stronie, a jego gradient maleje __eksponencjalnie__ szybko
  * jeśli aktywacje są daleko od własciwych, to gradient spada prawie do zera
  * wymnażanie bardzo małych wartosci tylko eksponencjalnie szybko je jeszcze zmniejsza...
3. eksplodujący gradient pojawia się równie często
  * jest efektem kilku wysokich aktywacji
  * może prowadzić do oscylacji, gdy nadchodzące sygnały są sprzeczne
  * w miarę łatwo sobie z nim poradzić przez obcinanie gradientu z wysoką normą
4. a jak z zanikającym gradientem?
  * trudniej: sieć nie uczy się wale albo potrzebuje wykładniczo wiele czasu
  * poprawna inicjalizacja
  * ReLU zamiast funkcji sigmoidalnych

# Problemy
1. __krótka__ a __długa__ pamięć
  * RNN z algorytmem typu BPTT szybko ___zapomina___ informacje
  * korzysta tylko z ostatniej
  * model dla angielskiego na poziomie znaków szybko nauczy się, że po znaku `q` __zawsze__ występuje znak `u`
  * jednak nie nauczy się informacji kontekstowej z poprzedniego zdania
2. wbrew pozorom można łatwo nauczyć model generowania zdań, patrz np. [prosty model Andreya Karpathiego](https://gist.github.com/karpathy/d4dee566867f8291f086)